In [248]:
import time
import numpy as np
import pinocchio as pin
from example_robot_data import load
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


In [249]:
robot = load('ur5')
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/


In [138]:
print(robot.data.oMi)

In [240]:
for i in range(robot.model.nframes):
    frame = robot.model.frames[i]
    print(f"Frame {i}: {frame.name}")

Frame 0: universe
Frame 1: root_joint
Frame 2: world
Frame 3: world_joint
Frame 4: base_link
Frame 5: base_link-base_fixed_joint
Frame 6: base
Frame 7: shoulder_pan_joint
Frame 8: shoulder_link
Frame 9: shoulder_lift_joint
Frame 10: upper_arm_link
Frame 11: elbow_joint
Frame 12: forearm_link
Frame 13: wrist_1_joint
Frame 14: wrist_1_link
Frame 15: wrist_2_joint
Frame 16: wrist_2_link
Frame 17: wrist_3_joint
Frame 18: wrist_3_link
Frame 19: ee_fixed_joint
Frame 20: ee_link
Frame 21: wrist_3_link-tool0_fixed_joint
Frame 22: tool0


In [241]:
for i in range(robot.model.njoints):
    joint = robot.model.joints[i]
    joint_name = robot.model.names[i]
    print(f"Joint {i}: {joint_name}")

Joint 0: universe
Joint 1: shoulder_pan_joint
Joint 2: shoulder_lift_joint
Joint 3: elbow_joint
Joint 4: wrist_1_joint
Joint 5: wrist_2_joint
Joint 6: wrist_3_joint


In [242]:
viz.display(pin.neutral(robot.model))

In [179]:
viz.display(pin.randomConfiguration(robot.model))

In [181]:
IDX_BASE = robot.model.getFrameId('base_link')
IDX_TOOL = robot.model.getFrameId('tool0')

In [182]:
for name, oMi in zip(robot.model.names, robot.data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}"
          .format( name, *oMi.translation.T.flat )))

universe                 :  0.00  0.00  0.00
shoulder_pan_joint       :  0.00  0.00  0.09
shoulder_lift_joint      : -0.13 -0.04  0.09
elbow_joint              :  0.11 -0.40  0.16
wrist_1_joint            :  0.19 -0.64 -0.15
wrist_2_joint            :  0.10 -0.67 -0.15
wrist_3_joint            :  0.09 -0.63 -0.23


In [183]:
robot.model.frames[IDX_TOOL]

Frame name: tool0 paired to (parent joint/ previous frame)(6/21)
with relative placement wrt parent joint:
  R =
          1           0           0
          0 4.89664e-12           1
          0          -1 4.89664e-12
  p =      0 0.0823      0
containing inertia:
  m = 0
  c = 0 0 0
  I = 
0 0 0
0 0 0
0 0 0

In [243]:
pin.framesForwardKinematics(robot.model, robot.data, q)

oMtool = robot.data.oMf[IDX_TOOL]
oMbase = robot.data.oMf[IDX_BASE]

print("Tool placement:",oMtool)
print("Basis placement:",oMbase)

Tool placement:   R =
 0.155811  0.107831  0.981884
  -0.9553 -0.236384  0.177553
 0.251247 -0.965658   0.06618
  p = 0.185502 0.400361 0.707798

Basis placement:   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0



In [ ]:
# given that forward kinematics is solved give as an exercise to find
# all relative transformations between the joints
# then as a test you can see whether or not your relative transformations
# end up with the same forward kinematics as the pin algorithm

IDX_BASE = robot.model.getFrameId('base_link')
IDX_SHOULDER_PAN = robot.model.getFrameId('shoulder_pan_joint')
IDX_SHOULDER_LIFT = robot.model.getFrameId('shoulder_lift_joint')
IDX_ELBOW = robot.model.getFrameId('base_link')
IDX_WRIST1 = robot.model.getFrameId('base_link')
IDX_WRIST2 = robot.model.getFrameId('base_link')
IDX_WRIST3 = robot.model.getFrameId('base_link')
IDX_TOOL = robot.model.getFrameId('tool0')

In [250]:
viz.display(pin.randomConfiguration(robot.model))

In [197]:
vq = np.random.rand(robot.model.nv)*4 - 2
DT = 1e-3
for t in range(1000):
    q = pin.integrate(robot.model,q,vq*DT)
    viz.display(q)
    time.sleep(DT/10)

In [252]:
oMgoal = pin.SE3(pin.Quaternion(-0.5, 0.58, -0.39, 0.52).normalized().matrix(), np.array([0.2, .4, .7]))
viz.addBox('goal', [.1,.1,.1], [ .1,.1,.5, .6])
viz.applyConfiguration('goal',oMgoal)

In [254]:
q = pin.neutral(robot.model)
viz.display(q)

for i in range(4000):  # Integrate over 2 second of robot life

    # Run the algorithms that outputs values in robot.data
    pin.framesForwardKinematics(robot.model,robot.data,q)
    pin.computeJointJacobians(robot.model,robot.data,q)

    # Placement from world frame o to frame f oMtool  
    oMtool = robot.data.oMf[IDX_TOOL]

    # 6D error between the two frame
    tool_nu = pin.log(oMtool.inverse()*oMgoal).vector

    # Get corresponding jacobian
    tool_Jtool = pin.computeFrameJacobian(robot.model, robot.data, q, IDX_TOOL, pin.LOCAL)

    # Control law by least square
    vq = np.linalg.pinv(tool_Jtool)@tool_nu

    q = pin.integrate(robot.model,q, vq * DT)
    viz.display(q)
